# PROYECTO INTEGRADOR

# ANÁLISIS DE SENTIMIENTOS EN RESEÑAS DE PROFESORES

# NOTEBOOK 2: MODELO CLÁSICO

Se utilizará **TF-IDF** para la vectorización y **Regresión Logística** y **Random Forest** como modelos predictivos.

## Preprocesamiento adicional

Los modelos clásicos requieren algunos pasos de preprocesamiento adicional que los modelos de deep learning no requieren. A continuación, se realizarán esos pasos para poder proceder con los modelados.


In [1]:
import pandas as pd

df = pd.read_csv("/kaggle/input/dataset-final-definitivo/dataset_final_definitivo.csv")

df.head()

,StudentComments,Sentiment
0,good,1
1,teacher,1
2,friendly teacher but not enough ability to enc...,1
3,he is a good techer.,1
4,he is agood techer.,1


### **Eliminación de stopwords**

Se eliminan las **stopwords** (palabras vacías) porque son términos muy comunes como “the”, “is” o “and” que aparecen con mucha frecuencia pero aportan poca información sobre el significado del texto. Al quitarlas, se reduce el ruido y se enfoca el análisis en las palabras más relevantes para tareas como clasificación o detección de sentimientos.


In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    if not isinstance(text, str):
        return ""
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

df['StudentComments'] = df['StudentComments'].astype(str).apply(remove_stopwords)

print(df.head())

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                     StudentComments  Sentiment
0                                               good          1
1                                            teacher          1
2  friendly teacher enough ability encourage student          1
3                                        good techer          1
4                                       agood techer          1


## Vectorización TF-IDF y Regresión Logística

La vectorización que se utilizará para los modelos clásicos será la **TF-IDF**.

El dataset se dividió en un conjunto de **training** y un conjunto de **test**. 

El primer modelo que se utilizará para realizar el análisis de sentimientos será la **regresión logística**. 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['StudentComments'])

y = df['Sentiment'].astype(int) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, multi_class='ovr')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8560911274130261
              precision    recall  f1-score   support

           0       0.71      0.42      0.52     14514
           1       0.87      0.96      0.92     61687

    accuracy                           0.86     76201
   macro avg       0.79      0.69      0.72     76201
weighted avg       0.84      0.86      0.84     76201



El modelo **Regresión Logística** alcanzó una accuracy del 85.6 %, lo cual indica que clasifica correctamente la polaridad de los comentarios en aproximadamente el 86 % de los casos. Aunque esta cifra refleja un buen rendimiento general, es fundamental considerar el **F1-score** para evaluar con mayor precisión el comportamiento del modelo en cada clase.

El F1-score para la clase negativa (0) es **0.52**, significativamente menor que el de la clase positiva (1), que alcanza 0.92. Esto evidencia un **rendimiento desequilibrado**, donde el modelo funciona mucho mejor para identificar comentarios positivos que negativos.

En la clase negativa, la precisión es de 0.71, lo que indica que, cuando el modelo predice que un comentario es negativo, generalmente acierta. Sin embargo, el recall es de apenas 0.42, lo que sugiere que muchos comentarios negativos no están siendo detectados. Por el contrario, en la clase positiva, el modelo muestra un recall muy alto (0.96) y una precisión también elevada (0.87), lo que significa que identifica correctamente la gran mayoría de los comentarios positivos, con pocos errores.

## Vectorización TF-IDF y Random Forest

El segundo modelo que se utilizará para realizar el análisis de sentimientos será **random forest**.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['StudentComments'])

y = df['Sentiment'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8245823545622761
              precision    recall  f1-score   support

           0       0.54      0.53      0.53     14514
           1       0.89      0.89      0.89     61687

    accuracy                           0.82     76201
   macro avg       0.71      0.71      0.71     76201
weighted avg       0.82      0.82      0.82     76201



El modelo de **Random Forest** alcanzó una accuracy del 82.5 %, lo que indica que clasifica correctamente aproximadamente el 82 % de los comentarios como positivos o negativos. Aunque esta métrica sugiere un buen rendimiento global, es importante analizar el **F1-score** por clase para comprender mejor el comportamiento del modelo, especialmente en datasets desbalanceados.

El F1-score para la clase negativa (0) es de **0.53**, mientras que para la clase positiva (1) es de 0.89, lo que evidencia una brecha significativa en el desempeño del modelo entre ambas clases. En otras palabras, el modelo es mucho más efectivo al identificar comentarios positivos que negativos.

En la clase negativa, tanto la precisión como el recall están en torno a 0.53, lo que indica un rendimiento limitado: el modelo acierta poco más de la mitad de las veces tanto al predecir comentarios negativos como al detectarlos. En contraste, para los comentarios positivos, el modelo logra una precisión y un recall altos (0.89 en ambos casos), lo que muestra una capacidad sólida para identificar correctamente la mayoría de los comentarios positivos.

## Cross-validation

Dado que el modelo de **Random Forest** obtuvo un mejor F1-Score que la Regresión Logística en la categoría negativa, que es la métrica en la que nos guiamos puesto que es la clase minoritaria, se selecciona como el mejor modelo clásico para continuar el proyecto.

Para asegurar una evaluación robusta, se aplicará el método de **cross-validation**, una técnica estadística que permite validar la estabilidad y generalización del modelo al entrenarlo y evaluarlo en diferentes subconjuntos de los datos. En este caso, se empleará la estrategia de **K-folds** con **5 pliegues**, lo que implica dividir el conjunto de datos en cinco partes y rotar la validación en cada una.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import numpy as np

tfidf = TfidfVectorizer(max_features=5000)

model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

pipeline = make_pipeline(tfidf, model)

X = df['StudentComments']
y = df['Sentiment'].astype(int)

scores = cross_val_score(pipeline, X, y, cv=5, scoring='f1_macro')

print("F1-score (macro) por fold:", scores)
print("F1-score (macro) promedio:", np.mean(scores))

F1-score (macro) por fold: [0.65456388 0.69775576 0.71876023 0.73141012 0.73763092]
F1-score (macro) promedio: 0.7080241828868828
